<a href="https://colab.research.google.com/github/Lee-Minsoo-97/Decision-Modeling/blob/main/Mark's_cafe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mark's Cafe

## Objective and Prerequisites

Mark's Cafe wants to maximize their profit by determining the number of Two-person tables(TT) and Four-person tables(FT) they should setup. The objectives of the Mark's Cafe problem are:

* The number of tables should be less than maximum tables that can be accomodated,
* Total serving and chef labour hours per hour cannot exceed the available, and
* The number of tables that can be setup are valid, i.e., non- negative values.

---
## Problem Description

Mark's Cafe wants to decide how many of each table type - two-person (TT) and four-person(FT) to setup at the cafe. Each TT generates a revenue of 100 dollars per hour and each FT generates 150 dollars. Labor is required for cooking and serving. In any given hour, there are 480 minutes of serving labor minutes and 300 minutes of chef labor minutes available. Each serving labor costs 18 dollars per hour and chef labor costs 32 dollars per hour. Each TT requires 10 minutes of serving labor and 20 minutes of chef labor, while each FT requires 15 minutes of serving labor and 35 minutes of chef labor. Mark wants to know how many of each table type he should accomodate such that his profit is maximized, but he cannot setup more than 6 two-person tables (TT) and 12 four-person tables (FT), and he cannot extend the labor hours.

## Model Formulation

### Indices

$i \in \{TT,FT\}$: Index to represent different table types.

### Parameters

$R_{i}$: Revenue per hour of table type $i$.

$CS$: Cost of serving hours.

$CC$: Cost of chef hours.

$LS_{i}$: Serving hours for table type $i$.

$LC_{i}$: Chef hours for table type $i$.

$AS$: Available serving time per hour.

$AC$: Available chef time per hour.

$N_{i}$: Maximum number of table type $i$.

### Decision Variables

$x_{i}$: Number of table type $i$ to set up .


### Objective Function

- **Profit**. We want to Maximize the total profit.


\begin{equation}
\text{Max}_{x_{i}} \quad \sum_{(i) \in {TT,FT}} [R_{i} - CS*LS_{i} -CC*LC_{i}]*x_{i}
\tag{0}
\end{equation}

### Constraints

\begin{equation}
\ x_{i} \leq N_{i} \quad \ i \in \{TT,FT\} \quad (\text{number of table type i cannot exceed maximum that can be accomodated})
\tag{1}
\end{equation}

\begin{equation}
\sum_{i \in (TT,FT)} LS_{i}*x_{i} \leq AS (\text{Serving hours cannot exceed available})
\tag{2}
\end{equation}

\begin{equation}
\sum_{i \in (TT,FT)} LC_{i}*x_{i} \leq AC (\text{Chef hours cannot exceed available})
\tag{3}
\end{equation}

\begin{equation}
\ x_{i} \geq 0 \quad \ i \in \{TT, FT\} \quad (\text{Non - Negative tables})
\tag{4}
\end{equation}


## Python Implementation

We now import the Gurobi Python Module

In [ ]:
pip install gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB

Set up the model

In [ ]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('mark\'s cafe')

In [ ]:
#Input Parameters
table_type = [*range(0,2)]
table_type_label =['TT','FT']

R = [100,150]   #Revenue per hour of table type i
CS = 18   #Cost of serving hours
CC = 32   #Cost of chef hours
LS = [0.167,0.25]   #Serving hours of table type i
LC = [0.333,0.583]  #Chef hours of table type i
AS = 8  #Available serving time per hour
AC = 5  #Available chef time per hour
N = [6,12]  #Maximum number of table type i

In [ ]:
#Decision Variable
x = m.addVars(table_type, vtype=GRB.CONTINUOUS, name=table_type_label)

In [ ]:
# Set the Maximize Obijective: Total profit
m.setObjective(gp.quicksum((R[i]-LS[i]*CS -LC[i]*CC)*x[i]
                          for i in table_type),  GRB.MAXIMIZE)

In [ ]:
# Total tables setup must be less than or equal to maximum that can be accomodated
c1 = m.addConstrs((x[i] <= N[i] for i in table_type), name = 'cannot exceed maximum tables')

# Serving hours cannot exceed available
c2 = m.addConstr((gp.quicksum(LS[i]*x[i] for i in table_type) <= AS), name = 'cannot exceed available serving hours')

# Chef hours cannot exceed available
c3 = m.addConstr((gp.quicksum(LC[i]*x[i] for i in table_type) <= AC), name ='cannot exceed available chef hours')

#Set Non-negative Decision Variable
c4 = m.addConstrs((x[i] >= 0 for i in table_type), name = 'non-negative number of tables')

Solve the model

In [ ]:
# Run the optimize solver
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 2 columns and 8 nonzeros
Model fingerprint: 0x0aca8e50
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [9e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 1e+01]
Presolve removed 6 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1711767e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.171176693e+03


Results of the Model

In [ ]:
# Get the Optimal Solution for X
print("The Optimal number of each table type to be set up for maximum profit\n")
m.printAttr('X')
def comma_value(number):
    return ("{:,}".format(number))
# print("\nTotal profit: $%8.2f" % (m.ObjVal))
print("\nTotal profit: $",comma_value(m.ObjVal)) #per hour
# m.printAttr('RHS')
# m.printAttr('Slack')


The Optimal number of each table type to be set up for maximum profit


    Variable            X 
-------------------------
          TT            6 
          FT      5.14923 

Total profit: $ 1,171.17669296741


More details on profits and resources used for each table type

In [ ]:
#Finding amount of Profit generated by each table type
e = m.getObjective()
margin = [e.getCoeff(i) for i in table_type ]
tables = [v.x for v in m.getVars()]
revenue = [margin[i]*tables[i] for i in table_type]
for i in table_type:
  print("Profit generated by",table_type_label[i],"table is $",comma_value(revenue[i]))

Profit generated by TT table is $ 518.028
Profit generated by FT table is $ 653.1486929674099


In [ ]:
#Finding the number of labor serving hours used by each table type
serving_hour_used = [LS[i]*tables[i] for i in table_type]
for i in table_type:
  print("Number of Serving hours used by",table_type_label[i],"table is",comma_value(serving_hour_used[i]))

print("Total Serving hours used is",comma_value(sum(serving_hour_used)))

Number of Serving hours used by TT table is 1.002
Number of Serving hours used by FT table is 1.2873070325900515
Total Serving hours used is 2.2893070325900515


In [ ]:
#Finding the number of chef hours used by each table type
chef_hour_used = [LC[i]*tables[i] for i in table_type]
for i in table_type:
  print("Number of chef hours used by",table_type_label[i],"table is",comma_value(chef_hour_used[i]))

print("Total chef hours",comma_value(sum(chef_hour_used)))

Number of chef hours used by TT table is 1.9980000000000002
Number of chef hours used by FT table is 3.002
Total chef hours 5.0


##  Conclusion

In the above problem we determined the number of tables that need to be setup at Mark's Cafe to obtain the maximum profit keeping in mind the maximum tables, serving and chef hour constarints.
It is seen that the number of tables to be set up are 6 for Two-person type and 5.14 (~6) for the Four-person type


##  References
[1] Sixty examples of business optimization models. https://ytyimin.github.io/tart-cherry/.

[2] Gurobi python reference. https://www.gurobi.com/documentation/